In [73]:
path='C:/Python/'

In [74]:
from __future__ import print_function
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import pandas as pd
import numpy
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import re

In [75]:
import pandas as pd
df=pd.read_csv(path+"天成飯店評論斷詞_feature.csv", index_col=None)
df.head(6)

,Unnamed: 0,Unnamed: 0.1,Title,Reviews,token,Rating,Place,Total_Post,Vote,Date_of_Stay,feature
0,0,0,很讚,謝謝天成大飯店的全體同仁，因為你們的努力，才有那麼好的住宿體驗！地點方便、服務人員又專業且體...,謝謝 天成 大飯店 的 全體同仁 因為 你們 的 努力 才 有 那麼 好 的 住宿 體驗 地...,50,no_place,1 篇投稿,no_helpful,2019年10月,人員 住宿 你們 值得 全體同仁 努力 因為 地點 大飯店 天成 專業 推薦 服務 謝謝 那...
1,1,1,地點方便,地點離捷運站近，要去台北景點都方便服務人員熱心，有問題也都能即時解答房間乾淨明亮，設備俱全，...,地點 離 捷運 站 近 要 去 台北 景點 都 方便 服務 人員 熱心 有 問題 也 都 能...,50,no_place,1 篇投稿,no_helpful,2019年10月,乾淨 人員 俱全 即時 台北 問題 地點 安靜 捷運 明亮 景點 服務 熱心 舒適 解答 設備
2,2,2,服務人員態度超好,離火車站、捷運很近櫃檯人員都很親切的幫忙解決問題一個人出來住到這裡真是太好了有機會一定會再來...,離 火車站 捷運 很近 櫃檯 人員 都 很 親切 的 幫忙 解決問題 一個 人 出來 住 到...,50,no_place,3 篇投稿,no_helpful,2019年10月,一個 一定 人員 再來 出來 別人 太好了 幫忙 很近 捷運 推薦 機會 櫃檯 火車站 真是...
3,3,3,推薦大家一家好飯店！,飯店到台北車站（火車、捷運、高鐵）非常便利，只要到台北出差都會選擇天成大飯店。 這是為什麼...,飯店 到 台北 車站 火車 捷運 高鐵 非常 便利 只要 到 台北 出差 都 會 選擇 天成...,50,台灣台北,10 篇投稿,no_helpful,2019年10月,一樣 一點 不然 人員 什麼 入住 兩下 出差 只要 可愛 台北 大力 她點 好吃 幫忙 愉...
4,4,4,休息一下,在臺北過夜一宿，環境舒適，服務人員相當棒，可以為您服務，住在這裏很方便，搭配捷運線想去哪都很...,在 臺北 過夜 一宿 環境 舒適 服務 人員 相當 棒 可以 為 您 服務 住 在 這裏 很...,50,台灣中正區,1 篇投稿,no_helpful,2019年10月,一宿 人員 便利 可以 夜景 捷運 搭配 晚上 服務 火車站 環境 相當 看到 線想 臺北 ...
5,5,5,颱風來襲也不用怕,"天成是台北車站附近的最佳飯店, 不用帶雨傘也可到達車站, 還有便利店在樓下, 完全不用擔心不...","天成 是 台北 車站 附近 的 最佳 飯店 , 不用 帶 雨傘 也 可 到達 車站 , ...",40,香港,532 篇投稿,16 人推薦,2019年9月,不同 不用 今次訂 來台 便利店 到達 多款 天成 完全 實在 床房 必會 擔心 旅行 最佳...


In [196]:
# 無 feature 跳過
del_set=[]
for n in range(len(list(df.feature))):
    if type(list(df.feature)[n])!=float:
        if len(list(df.feature)[n])==0:
            del_set.append(n)      
    else:
        del_set.append(n)
print (del_set)
df.drop(axis=1, index=del_set, inplace=True)
# 重新編碼
df = df.reset_index(drop=True)
len(df)    

[]


1485

In [6]:
pip install gensim

  Stored in directory: C:\Users\loy07\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [77]:
from gensim import corpora
from gensim import models
from collections import defaultdict
from gensim import similarities
from pprint import pprint 

In [197]:
texts=list(df.feature) #######################################選取文本

# 刪除只出現一次的字詞，巨量文本分析，極少出現的字，代表不重要
frequency = defaultdict(int)
texts2=[]
for text in texts:
    if type(text) !=float:
        texts2.append(text)
    
for text in texts2:    
    text=text.split()
    for token in text:
        frequency[token] += 1            
texts_list = [[token for token in text.split() if frequency[token] > 1] for text in texts2]  

In [198]:
# 各篇文章文字袋
print (texts_list[0])

['人員', '住宿', '你們', '值得', '努力', '因為', '地點', '大飯店', '天成', '專業', '推薦', '服務', '謝謝', '那麼', '體貼', '體驗']


In [199]:
# 引數 texts_list, 為各篇文章的文字袋 list 
dictionary = corpora.Dictionary(texts_list)
print(dictionary)

Dictionary(2947 unique tokens: ['人員', '住宿', '你們', '值得', '努力']...)


In [200]:
# 顯示dictionary 的字詞、編碼對應
dictionary.compactify()
print(dictionary.token2id)

{'人員': 0, '住宿': 1, '你們': 2, '值得': 3, '努力': 4, '因為': 5, '地點': 6, '大飯店': 7, '天成': 8, '專業': 9, '推薦': 10, '服務': 11, '謝謝': 12, '那麼': 13, '體貼': 14, '體驗': 15, '乾淨': 16, '俱全': 17, '即時': 18, '台北': 19, '問題': 20, '安靜': 21, '捷運': 22, '明亮': 23, '景點': 24, '熱心': 25, '舒適': 26, '解答': 27, '設備': 28, '一個': 29, '一定': 30, '再來': 31, '出來': 32, '別人': 33, '太好了': 34, '幫忙': 35, '很近': 36, '機會': 37, '櫃檯': 38, '火車站': 39, '真是': 40, '親切': 41, '解決問題': 42, '這裡': 43, '一樣': 44, '一點': 45, '不然': 46, '什麼': 47, '入住': 48, '出差': 49, '只要': 50, '可愛': 51, '大力': 52, '好吃': 53, '愉快': 54, '會議': 55, '火車': 56, '狀況': 57, '經驗': 58, '聰明': 59, '謝謝你們': 60, '這次': 61, '還好': 62, '順利': 63, '飯店': 64, '高鐵': 65, '便利': 66, '可以': 67, '夜景': 68, '搭配': 69, '晚上': 70, '環境': 71, '相當': 72, '看到': 73, '臺北': 74, '這裏': 75, '過夜': 76, '不同': 77, '不用': 78, '來台': 79, '便利店': 80, '到達': 81, '多款': 82, '完全': 83, '實在': 84, '擔心': 85, '旅行': 86, '最佳': 87, '樓下': 88, '空間': 89, '組合': 90, '足夠': 91, '車站': 92, '適合': 93, '選擇': 94, '還有': 95, '附近': 96, '雨傘': 97, '上次': 98, '十分滿意': 99,

In [201]:
# 查某一個 vlue 的 key 值
ind=[i[0] for i in dictionary.items() if i[1]=='櫃台'][0]
print (ind)

333


In [202]:
corpus = [dictionary.doc2bow(text) for text in texts_list]

In [203]:
len(corpus)
print (corpus[10])

[(0, 1), (6, 1), (11, 1), (40, 1), (43, 1), (49, 1), (157, 1), (158, 1), (159, 1), (160, 1), (161, 1), (162, 1), (163, 1), (164, 1), (165, 1)]


In [204]:
# 儲存字典。
dictionary.save(path+ 'dictionary.dict') 
# 儲存 corpus。
corpora.MmCorpus.serialize(path+ 'corpus.mm', corpus)  

In [205]:
############################################################################
filename=path+ 'dictionary.dict'
corpusname=path+ 'corpus.mm'

# 暫存檔的位置
if os.path.isfile(filename):
    dictionary = corpora.Dictionary.load(filename)
    corpus = corpora.MmCorpus(corpusname)    
    print ( filename +', ' + corpusname)
else:
    print("can't find these two files")

C:/Python/dictionary.dict, C:/Python/corpus.mm


In [206]:
# 引數為 corpus
tfidf = models.TfidfModel(corpus, normalize=True) # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]

In [207]:
corpus_tfidf[0]

[(0, 0.14381652613992962),
 (1, 0.1852845168853241),
 (2, 0.37780912678475265),
 (3, 0.20364853673053704),
 (4, 0.3197171633548431),
 (5, 0.23502836730335486),
 (6, 0.16917805705317718),
 (7, 0.1990064157757166),
 (8, 0.14227331079859767),
 (9, 0.26841194045517225),
 (10, 0.16975256513414771),
 (11, 0.13539658983178005),
 (12, 0.29281241710026357),
 (13, 0.3318151592716032),
 (14, 0.35548242106587336),
 (15, 0.27101444142063436)]

In [208]:
# 如果不加權 corpus
corpus[0]

[(0, 1.0),
 (1, 1.0),
 (2, 1.0),
 (3, 1.0),
 (4, 1.0),
 (5, 1.0),
 (6, 1.0),
 (7, 1.0),
 (8, 1.0),
 (9, 1.0),
 (10, 1.0),
 (11, 1.0),
 (12, 1.0),
 (13, 1.0),
 (14, 1.0),
 (15, 1.0)]

In [209]:
# transform corpus to LSI |space and index it
index = similarities.MatrixSimilarity(lsi[corpus_tfidf]) 
print (index)

MatrixSimilarity<1485 docs, 30 features>


In [21]:
#潛在語意分析

In [210]:
ntopics=30
# corpus_tfidf 為文字詞袋，經過 tfidf轉換, id2word 字詞的代碼與名稱
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=ntopics) 
# create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
corpus_lsi = lsi[corpus_tfidf] 
term_proj=lsi.projection.u

In [211]:
lsi.print_topics(ntopics)

[(0,
  '0.152*"乾淨" + 0.147*"車站" + 0.145*"服務" + 0.140*"非常" + 0.140*"台北" + 0.138*"交通" + 0.134*"方便" + 0.133*"下次" + 0.133*"位置" + 0.132*"臺北"'),
 (1,
  '0.215*"交通" + 0.207*"台北" + -0.190*"友好" + 0.186*"飯店" + -0.181*"臺北" + 0.179*"人員" + 0.172*"親切" + -0.171*"工作人員" + -0.167*"一個" + -0.153*"這家"'),
 (2,
  '-0.320*"房间" + -0.310*"服务" + -0.254*"车站" + -0.232*"捷运" + -0.188*"干净" + -0.172*"不错" + -0.170*"热情" + -0.162*"人员" + -0.143*"选择" + -0.141*"旁边"'),
 (3,
  '-0.215*"位置" + -0.183*"乾淨" + -0.171*"還會" + -0.155*"友好" + -0.154*"車站" + -0.153*"方便" + -0.144*"再來" + -0.128*"工作人員" + -0.120*"下次" + 0.119*"我們"'),
 (4,
  '0.252*"再來" + 0.244*"還會" + -0.206*"車站" + 0.179*"下次" + -0.157*"機場" + -0.153*"步行" + -0.151*"附近" + -0.150*"分鐘" + -0.149*"出口" + -0.144*"旁邊"'),
 (5,
  '-0.281*"人員" + 0.251*"整潔" + -0.250*"服務" + 0.205*"員工" + -0.172*"早餐" + 0.155*"友善" + 0.152*"服務員" + -0.151*"火車站" + 0.148*"入住" + 0.133*"朋友"'),
 (6,
  '-0.341*"天成" + -0.240*"推薦" + -0.224*"大飯店" + 0.191*"整潔" + 0.190*"舒適" + 0.176*"再來" + 0.159*"地點" + 0.158*"友善" + -0.157*"值

In [212]:
print(lsi)

LsiModel(num_terms=2947, num_topics=30, decay=1.0, chunksize=20000)


In [213]:
# 最前面的20個字詞
lsi.print_topic(0, topn=20)

'0.152*"乾淨" + 0.147*"車站" + 0.145*"服務" + 0.140*"非常" + 0.140*"台北" + 0.138*"交通" + 0.134*"方便" + 0.133*"下次" + 0.133*"位置" + 0.132*"臺北" + 0.132*"酒店" + 0.128*"舒適" + 0.126*"工作人員" + 0.122*"飯店" + 0.120*"再來" + 0.116*"火車站" + 0.116*"人員" + 0.116*"天成" + 0.114*"友好" + 0.113*"還會"'

In [214]:
# 字詞的權數，權數愈大表示在 topic 中愈重要。
# topic number=0, top number=5 
lsi.show_topic(0,topn=20)

[('乾淨', 0.15153598876696495),
 ('車站', 0.14707816458103018),
 ('服務', 0.14496222864375907),
 ('非常', 0.1400229972473197),
 ('台北', 0.13952384932505654),
 ('交通', 0.13828564171525196),
 ('方便', 0.133514556993172),
 ('下次', 0.1334569912024876),
 ('位置', 0.13310267852473737),
 ('臺北', 0.132302687077489),
 ('酒店', 0.1316550549030464),
 ('舒適', 0.12753942173320046),
 ('工作人員', 0.1255778086349353),
 ('飯店', 0.12185977003120568),
 ('再來', 0.1198398690916734),
 ('火車站', 0.1162148367236193),
 ('人員', 0.1160207892592181),
 ('天成', 0.11575345833427851),
 ('友好', 0.11392276023516656),
 ('還會', 0.11255185927253819)]

In [215]:
for n in range(ntopics):
    keywords=[]
    for m in range(len(lsi.show_topic(n,topn=30))):
        keywords.append(lsi.show_topic(n,topn=30)[m][0])
    key=",".join(keywords)    
    print (n, key)

0 乾淨,車站,服務,非常,台北,交通,方便,下次,位置,臺北,酒店,舒適,工作人員,飯店,再來,火車站,人員,天成,友好,還會,早餐,這裏,捷運,不錯,員工,這家,選擇,入住,旁邊,我們
1 交通,台北,友好,飯店,臺北,人員,親切,工作人員,一個,這家,地點,捷運,這裏,酒店,入住,樂於助人,天成,態度,整潔,方便,下次,服務,我們,地鐵站,舒適,這是,宇宙,他們,十分,很棒
2 房间,服务,车站,捷运,干净,不错,热情,人员,选择,旁边,饭店,升级,火车站,还会,这里,我们,设施,免费,工作人员,感觉,整洁,推荐,台湾,大饭店,舒适,还有,地理位置,服務,出行,购物
3 位置,乾淨,還會,友好,車站,方便,再來,工作人員,下次,我們,臺北,沒有,火車站,他們,感覺,非常,行李,這裏,服務員,覺得,問題,雖然,有點,舒適,但是,服務,可能,因為,要求,房間
4 再來,還會,車站,下次,機場,步行,附近,分鐘,出口,旁邊,人員,方便,朋友,台北,一定,推薦,捷運,服務,機會,喜歡,巴士,很多,肯定,地鐵,購物,位於,這裏,再次,很棒,入住
5 人員,整潔,服務,員工,早餐,友善,服務員,火車站,入住,朋友,很棒,不錯,親切,服務態度,禮貌,大飯店,再次,非常,天成,下次,職員,飯店,十分,旁邊,一定,很近,有禮,酒店,便利,好吃
6 天成,推薦,大飯店,整潔,舒適,再來,地點,友善,值得,員工,車站,台北,朋友,分鐘,鄰近,第一次,還會,服務態度,房間,火車站,寬敞,旁邊,良好,大酒店,真的,適中,台灣,餐廳,很大,選擇
7 選擇,下次,再來,還會,附近,親切,車站,服務,人員,乾淨,很多,態度,交通,容易,天成,出口,購物,找到,友好,台灣,推薦,真的,早餐,還有,感覺,大飯店,機會,服務態度,肯定,工作人員
8 早餐,乾淨,房間,不錯,捷運,親切,容易,車站,人員,禮貌,自助,出口,購物,地方,台北,有點,步行,火車站,找到,沒有,位置,但是,再來,餐廳,值得,不過,高鐵,方便,住宿,附近
9 推薦,朋友,方便,出口,可以,地方,旁邊,下次,購物,附近,車站,還會,分鐘,免費,就是,再來,家人,值得,熱情,不錯,喜歡,服務員,禮貌,交通,選擇,臺北,入住,價格,客房,不過
10 有點,雖然,捷運,但是,早餐,出口,地理位置,工作人員,服務,就是,旁邊,舒服,還

In [216]:
print(corpus_lsi[0])

[(0, 0.15651850718445312), (1, 0.13978172823314555), (2, 0.03829163819854373), (3, 0.06107713644858872), (4, 0.11785884270116848), (5, -0.02354784542850277), (6, -0.18248548118772098), (7, -0.04413257641927501), (8, 0.0935147510567061), (9, 0.031080355611505413), (10, -0.012103371727361608), (11, 0.009099178413931437), (12, -0.028690295639204954), (13, -0.026093383215796368), (14, 0.002672065268077302), (15, 0.06735798793272672), (16, 0.03616831834454468), (17, 0.07931557270861483), (18, -0.0332184749902364), (19, -0.09546541859490032), (20, -0.02415374535114029), (21, -0.024222475898453312), (22, 0.03294951290745212), (23, 0.010277947262903092), (24, 0.013850744150427302), (25, -0.01664877943464767), (26, -0.03516154540184655), (27, 0.09519983150747535), (28, 0.057132171142685824), (29, -0.06985328540753977)]


In [217]:
key='服務'
rec=[]
count=[]
for n in range(len(df.feature)):
    c=df.feature[n].count(key)
    if c>0:
        rec.append(n)
        count.append(c)
print (rec)
print (count)

[0, 1, 3, 4, 6, 10, 14, 16, 18, 20, 21, 23, 24, 28, 33, 38, 39, 42, 45, 47, 54, 57, 61, 62, 63, 67, 71, 74, 77, 79, 80, 83, 89, 90, 91, 96, 98, 111, 115, 118, 124, 125, 127, 129, 131, 132, 136, 137, 139, 140, 144, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 164, 165, 166, 168, 169, 174, 176, 182, 184, 186, 189, 191, 193, 194, 195, 196, 197, 198, 200, 202, 215, 219, 224, 226, 227, 228, 230, 238, 239, 247, 248, 250, 255, 256, 257, 259, 260, 261, 263, 265, 267, 269, 273, 274, 275, 279, 286, 287, 288, 290, 291, 300, 302, 304, 305, 308, 323, 325, 331, 332, 333, 334, 335, 337, 343, 345, 348, 349, 350, 360, 366, 367, 369, 370, 377, 378, 387, 391, 394, 396, 397, 401, 403, 405, 406, 407, 411, 412, 413, 414, 415, 417, 418, 419, 425, 426, 429, 431, 433, 435, 436, 439, 440, 442, 447, 449, 450, 452, 454, 458, 459, 463, 464, 465, 467, 468, 470, 471, 473, 474, 475, 477, 479, 481, 485, 488, 489, 491, 493, 500, 502, 503, 504, 506, 507, 508, 509, 511, 514, 522, 524, 525, 529, 533, 5

In [218]:
doc_list=list(df.Title)

In [219]:
# 任選一篇文件，在topics 空間上的投影
d=rec[0]   #################################
corpus_tfidf[d]
vec_lsi0 = lsi[corpus_tfidf[d]] # convert the query to LSI space

# print (doc_list[d])
# print(vec_lsi0)

# 該篇文章最接近的排列
sims = index[vec_lsi0] # perform a similarity query against the corpus
# 依最接近的排列
sims = sorted(enumerate(sims), key=lambda item: -item[1])
# print(sims) # print sorted (document number, similarity score) 2-tuples  

# 前十名
print (sims[0][0],doc_list[d])
print ('---------')
for n in range(1,20):
    print (sims[n][0],doc_list[int(sims[n][0])],'相似度: ',sims[n][1])

1358 很讚
---------
1254 3天2晚住宿在臺北天成飯店  相似度:  0.62978506
1415 最佳選擇 相似度:  0.6217615
1370 出色的客戶服務 相似度:  0.61974657
1191 很棒的位置和樂於助人的員工 相似度:  0.61339724
1146 很棒的乾淨的酒店和一個很棒的位置 相似度:  0.60240936
1108 很棒的住宿！ 相似度:  0.59430236
1144 輝煌歷程... 相似度:  0.59348655
1116 細心&樂於助人的員工  相似度:  0.58886933
971 感覺像在家一樣 相似度:  0.5878055
954 一個很棒的住宿 相似度:  0.5862187
1263 強烈推薦 相似度:  0.5799862
1391 酒店位於一個良好的位置  相似度:  0.57724905
1447 方便, 實惠而舒適的 相似度:  0.5741509
1240 從來沒有不舒服 相似度:  0.5728631
1454 鞋盒大男孩位置很棒的服務 相似度:  0.56615925
214 地點很方便，工作人員很熱情 相似度:  0.56020457
1124 優質的服務 相似度:  0.5592143
19 GOOOOOOD 相似度:  0.55424374
1158 很棒的服務！ 相似度:  0.5521196


In [220]:
import numpy as np
def cos_simility(a, b):
    # list 改 array
    x = np.array(a)
    y = np.array(b)
    # 向量長度
    Lx=np.linalg.norm(x)
    Ly=np.linalg.norm(y)
    # cos 值
    cos=round(x.dot(y)/(Lx*Ly),8)
    return cos

# k 個數
# kw 關鍵字
# dictionary
# term_proj 字詞的投影
def kClosestTerms(k,kw,dictionary,term_proj): 
    index=[i[0] for i in dictionary.items() if i[1]==kw][0] 
    closestTerms = {}
    words=list(dictionary.items())
    terms=[]
    for n in words:
        terms.append(n[1])
    for i in range(len(terms)):
        if terms[i]!=kw:        
            closestTerms[terms[i]] = cos_simility(term_proj[index],term_proj[i])

    # 依 dictionary 的 value (即相似性) 排序
    sortedList = sorted(closestTerms , key= lambda l : closestTerms[l])

    # 取前 k個，字詞及相似性
    lis_sorted =sortedList[::-1][0:k]
    lis_sim=[]
    for l in lis_sorted:
        lis_sim.append(closestTerms[l])        

    data=list(zip(lis_sorted, lis_sim))   
    df=pd.DataFrame(data)      
    return df,data

In [221]:
kClosestTerms(20,"人員",dictionary,term_proj)[0]

,0,1
0,服務,0.581331
1,親切,0.467699
2,周遭,0.462705
3,旅行者,0.436648
4,時櫃,0.433932
5,透過,0.414883
6,清楚,0.411112
7,動作,0.410844
8,悠閒,0.404763
9,需求,0.404115


In [222]:
kw=['人員','位置','不錯','交通',"設備",'服務']
index=[]
for k in range(len(kw)):
    index.append([i[0] for i in dictionary.items() if i[1]==kw[k]][0])
similar=[]
for n in range(len(index)):
    sim=[]
    for m in range(len(index)):           
        similarity= cos_simility(term_proj[index[n]],term_proj[index[m]]) 
        sim.append(similarity)
    similar.append(sim)
df_similar=pd.DataFrame(similar,columns=kw, index=kw)    
df_similar    

,人員,位置,不錯,交通,設備,服務
人員,1.000000,-0.081498,0.014418,0.017091,0.047486,0.581331
位置,-0.081498,1.000000,0.102939,-0.019496,0.017213,0.159961
不錯,0.014418,0.102939,1.000000,0.190976,-0.041791,0.159605
交通,0.017091,-0.019496,0.190976,1.000000,0.204159,0.074101
設備,0.047486,0.017213,-0.041791,0.204159,1.000000,0.015261
服務,0.581331,0.159961,0.159605,0.074101,0.015261,1.000000


# LDA

In [170]:
from gensim import models
ntopics=10 #######################################################################
# 給定 corpus, id2word, num of topics
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=ntopics, minimum_probability=0.05)
# 執行轉換
lda_res = lda[corpus_tfidf]
# 如果不加權
#lda_res = lda[corpus]

In [166]:
ntopics=10 #######################################################################
# 給定 corpus, id2word, num of topics
lda1 = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=ntopics,passes=40, minimum_probability=0.05)
# 執行轉換
lda_res1 = lda1[corpus_tfidf]
# 如果不加權
#lda_res = lda[corpus]

In [171]:
#不加權
ntopics=10 #######################################################################
# 給定 corpus, id2word, num of topics
lda2 = models.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, minimum_probability=0.05)
# 執行轉換
lda_res2 = lda2[corpus]

In [172]:
print (type(lda_res2))
len(lda_res2)

<class 'gensim.interfaces.TransformedCorpus'>


1485

In [173]:
# 經過lda轉換，在各 tops 的位置。
# 注意是機率的概念，均為正值 
for doc in lda_res:
    print(doc)

[(2, 0.8123939)]
[(2, 0.8093741)]
[(6, 0.8201051)]
[(2, 0.8481352)]
[(9, 0.8065688)]
[(1, 0.83503044)]
[(2, 0.81361336)]
[(0, 0.8136801)]
[(9, 0.8469242)]
[(5, 0.818435)]
[(5, 0.80747503)]
[(9, 0.8323313)]
[(6, 0.81937325)]
[(2, 0.8133393)]
[(6, 0.8234517)]
[(1, 0.8168045)]
[(2, 0.78391594)]
[(9, 0.805893)]
[(2, 0.8203465)]
[(9, 0.8368459)]
[(6, 0.3481861), (8, 0.49040663)]
[(3, 0.8277238)]
[(3, 0.8178002)]
[(8, 0.81348675)]
[(3, 0.80542946)]
[(4, 0.7930335)]
[(0, 0.7992318)]
[(7, 0.79481524)]
[(3, 0.80650395)]
[(1, 0.81103104)]
[(0, 0.8518182)]
[(5, 0.8460045)]
[(4, 0.67883795), (9, 0.19256812)]
[(1, 0.8283812)]
[(9, 0.81033987)]
[(0, 0.8083043)]
[(7, 0.8450496)]
[(0, 0.8321178)]
[(0, 0.28166544), (1, 0.5650439)]
[(6, 0.8179575)]
[(6, 0.8198443)]
[(9, 0.82096267)]
[(0, 0.7998742)]
[(8, 0.8388896)]
[(1, 0.84908164)]
[(2, 0.80986005)]
[(4, 0.84340435)]
[(1, 0.8008582)]
[(4, 0.86376244)]
[(3, 0.807592)]
[(9, 0.8386885)]
[(1, 0.83724356)]
[(5, 0.8332411)]
[(2, 0.81341654)]
[(5, 0.80653155

[(1, 0.30733123), (9, 0.543391)]
[(6, 0.8263184)]
[(1, 0.53146446), (5, 0.2871747)]
[(3, 0.17401935), (5, 0.70005745)]
[(1, 0.80738366)]
[(2, 0.82774186)]
[(6, 0.82771933)]
[(9, 0.8348562)]
[(6, 0.81886756)]
[(8, 0.8173765)]
[(2, 0.8177667)]
[(2, 0.8203538)]
[(3, 0.82594705)]
[(0, 0.795989)]
[(5, 0.84732676)]
[(0, 0.7976384)]
[(5, 0.82769907)]
[(3, 0.81777567)]
[(2, 0.37524274), (3, 0.48785374)]
[(0, 0.23236816), (5, 0.6398065)]
[(0, 0.8607108)]
[(9, 0.82400095)]
[(9, 0.8003332)]
[(6, 0.8328034)]
[(3, 0.79382807)]
[(6, 0.8276292)]
[(5, 0.8424963)]
[(4, 0.8343432)]
[(9, 0.80767155)]
[(2, 0.7802747)]
[(8, 0.8137543)]
[(6, 0.7989085)]
[(6, 0.84339505)]
[(7, 0.8140582)]
[(2, 0.8410898)]
[(0, 0.8429791)]
[(4, 0.8375587)]
[(9, 0.8100055)]
[(3, 0.8234411)]
[(6, 0.8327059)]
[(8, 0.8047527)]
[(3, 0.8095953)]
[(8, 0.81966764)]
[(2, 0.8123169)]
[(2, 0.8201592)]
[(5, 0.8221291)]
[(5, 0.31421843), (9, 0.53117186)]
[(3, 0.83459485)]
[(1, 0.80968404)]
[(4, 0.8178698)]
[(1, 0.82299745)]
[(1, 0.7768054

[(1, 0.85233307)]
[(4, 0.8521534)]
[(8, 0.828095)]
[(1, 0.86136544)]
[(0, 0.86202854)]
[(3, 0.13226166), (8, 0.72818834)]
[(9, 0.84445834)]
[(1, 0.8450147)]
[(8, 0.8506438)]
[(7, 0.8058829)]
[(5, 0.861835)]
[(3, 0.81605655)]
[(6, 0.19235866), (7, 0.68533194)]
[(0, 0.8340382)]
[(9, 0.82070917)]
[(0, 0.549872), (4, 0.3141573)]
[(1, 0.8399799)]
[(1, 0.8342729)]
[(8, 0.868757)]
[(7, 0.847468)]
[(0, 0.8526736)]
[(1, 0.7940725)]
[(0, 0.8050317)]
[(3, 0.8618229)]
[(3, 0.86794424)]
[(2, 0.87045074)]
[(9, 0.8271604)]
[(7, 0.8552147)]
[(8, 0.8382545)]
[(8, 0.83682734)]
[(8, 0.79650635)]
[(6, 0.5988213), (8, 0.27740252)]
[(1, 0.81589407)]
[(8, 0.84312475)]
[(8, 0.8252732)]
[(3, 0.83797854)]
[(4, 0.85655165)]
[(6, 0.8492464)]
[(5, 0.8302457)]
[(8, 0.83204377)]
[(0, 0.8436846)]
[(5, 0.8095325)]
[(5, 0.82873803)]
[(3, 0.82568085)]
[(5, 0.8656302)]
[(8, 0.8574598)]
[(6, 0.86335194)]
[(3, 0.8452965)]
[(6, 0.84438)]
[(2, 0.86109275)]
[(6, 0.8217117)]
[(5, 0.85532016)]
[(3, 0.8538778)]
[(2, 0.8565873)]


[(0, 0.8344578)]
[(0, 0.8438805)]
[(6, 0.8572327)]
[(1, 0.8029445)]
[(4, 0.86477673)]
[(8, 0.8543774)]
[(2, 0.10393188), (3, 0.7726515)]
[(8, 0.8507831)]
[(9, 0.8382915)]
[(5, 0.85481274)]
[(7, 0.86640584)]
[(9, 0.8118033)]
[(2, 0.8597104)]
[(3, 0.87080556)]
[(6, 0.8408136)]
[(4, 0.86038023)]
[(7, 0.8392909)]
[(8, 0.8593784)]
[(7, 0.8175682)]
[(6, 0.8379416)]
[(8, 0.8670072)]
[(8, 0.87148046)]
[(1, 0.8568914)]
[(0, 0.20520547), (2, 0.6799242)]
[(4, 0.8384383)]
[(4, 0.8522815)]


In [174]:
# 顯示所有的 topics 有加權
top = lda.show_topics(20)
for t in top:
    print (t)

(0, '0.003*"非常" + 0.003*"就是" + 0.003*"酒店" + 0.003*"乾淨" + 0.003*"舒適" + 0.003*"台北" + 0.003*"附近" + 0.003*"這家" + 0.003*"臺北" + 0.003*"友好"')
(1, '0.004*"出口" + 0.003*"員工" + 0.003*"台北" + 0.003*"而且" + 0.003*"飯店" + 0.003*"推薦" + 0.003*"酒店" + 0.003*"交通" + 0.003*"天成" + 0.003*"可以"')
(2, '0.004*"酒店" + 0.004*"這家" + 0.004*"服務" + 0.004*"交通" + 0.004*"這是" + 0.004*"火車站" + 0.004*"可以" + 0.004*"車站" + 0.003*"方便" + 0.003*"我們"')
(3, '0.004*"車站" + 0.004*"台北" + 0.004*"入住" + 0.004*"下次" + 0.004*"舒適" + 0.003*"服務" + 0.003*"酒店" + 0.003*"整潔" + 0.003*"飯店" + 0.003*"方便"')
(4, '0.003*"服務" + 0.003*"臺北" + 0.003*"我們" + 0.003*"酒店" + 0.003*"非常" + 0.003*"房間" + 0.003*"交通" + 0.003*"下次" + 0.003*"飯店" + 0.003*"人員"')
(5, '0.003*"台北" + 0.003*"他們" + 0.003*"這家" + 0.003*"一個" + 0.003*"捷运" + 0.003*"酒店" + 0.003*"一次" + 0.003*"地鐵站" + 0.003*"台灣" + 0.002*"服务"')
(6, '0.005*"捷運" + 0.004*"交通" + 0.004*"人員" + 0.004*"飯店" + 0.004*"服務" + 0.003*"天成" + 0.003*"乾淨" + 0.003*"非常" + 0.003*"禮貌" + 0.003*"入住"')
(7, '0.004*"臺北" + 0.004*"非常" + 0.004*"友好" + 0.004*"工作

In [175]:
# 顯示所有的 topics 有加權 & passes
top = lda1.show_topics(20)
for t in top:
    print (t)

(0, '0.004*"如果" + 0.004*"左右" + 0.003*"路程" + 0.003*"同事" + 0.003*"好離" + 0.003*"難忘" + 0.002*"桃園" + 0.002*"行李" + 0.002*"周邊" + 0.002*"公共交通"')
(1, '0.014*"房间" + 0.013*"服务" + 0.011*"车站" + 0.010*"捷运" + 0.009*"干净" + 0.007*"热情" + 0.007*"不错" + 0.007*"人员" + 0.006*"旁边" + 0.006*"选择"')
(2, '0.004*"那裡" + 0.003*"算是" + 0.003*"美中不足" + 0.003*"隔音" + 0.003*"隔壁" + 0.003*"我來" + 0.003*"這兒" + 0.003*"加強" + 0.003*"當地" + 0.002*"如果"')
(3, '0.005*"我們" + 0.005*"如果" + 0.004*"強烈推薦" + 0.004*"大酒店" + 0.003*"一個" + 0.003*"退房" + 0.003*"沒有" + 0.003*"手續" + 0.003*"一次" + 0.003*"房間"')
(4, '0.005*"擔心" + 0.003*"大酒店" + 0.003*"坐火車" + 0.003*"不用" + 0.003*"十分滿意" + 0.002*"人房" + 0.002*"當天" + 0.002*"東西" + 0.002*"走路" + 0.002*"你們"')
(5, '0.006*"酒店" + 0.006*"臺北" + 0.006*"服務" + 0.006*"火車站" + 0.006*"車站" + 0.006*"位置" + 0.006*"乾淨" + 0.005*"舒適" + 0.005*"這家" + 0.005*"台北"')
(6, '0.003*"其他人" + 0.003*"差不多" + 0.002*"目的地" + 0.002*"樓層" + 0.002*"禮物" + 0.002*"商场" + 0.002*"美妙" + 0.002*"那個" + 0.002*"有善" + 0.002*"毛巾"')
(7, '0.003*"高铁" + 0.003*"楼下" + 0.003*"蛋糕

In [176]:
# 顯示所有的 topics 無加權
top = lda2.show_topics(20)
for t in top:
    print (t)

(0, '0.015*"台北" + 0.011*"舒適" + 0.011*"乾淨" + 0.010*"飯店" + 0.010*"早餐" + 0.010*"車站" + 0.010*"酒店" + 0.010*"我們" + 0.009*"交通" + 0.009*"服務"')
(1, '0.016*"酒店" + 0.010*"我們" + 0.010*"臺北" + 0.009*"可以" + 0.008*"一個" + 0.008*"這家" + 0.007*"台北" + 0.007*"飯店" + 0.006*"非常" + 0.006*"車站"')
(2, '0.011*"飯店" + 0.009*"酒店" + 0.008*"我們" + 0.007*"工作人員" + 0.007*"捷運" + 0.007*"下次" + 0.007*"一個" + 0.006*"入住" + 0.006*"非常" + 0.006*"方便"')
(3, '0.010*"一個" + 0.007*"台北" + 0.007*"酒店" + 0.005*"這裏" + 0.005*"十分" + 0.004*"我們" + 0.004*"房間" + 0.004*"附近" + 0.004*"設施" + 0.004*"還有"')
(4, '0.012*"酒店" + 0.008*"工作人員" + 0.008*"很棒" + 0.007*"友好" + 0.007*"一個" + 0.007*"地方" + 0.007*"我們" + 0.007*"非常" + 0.006*"他們" + 0.006*"位置"')
(5, '0.012*"酒店" + 0.009*"我們" + 0.008*"一個" + 0.008*"臺北" + 0.007*"再來" + 0.007*"這裏" + 0.007*"員工" + 0.006*"而且" + 0.006*"飯店" + 0.006*"舒適"')
(6, '0.011*"我們" + 0.008*"服務" + 0.008*"人員" + 0.007*"交通" + 0.005*"酒店" + 0.005*"地理位置" + 0.005*"一個" + 0.005*"有點" + 0.005*"行李" + 0.005*"房間"')
(7, '0.015*"酒店" + 0.011*"台北" + 0.008*"非常" + 0.007

In [178]:
# 第 n 個 topic
n=9
top = lda.show_topic(n,20)
for t in top:
    print (t)

('台北', 0.003565425)
('這裏', 0.003362992)
('酒店', 0.0032762059)
('乾淨', 0.003262736)
('下次', 0.0032562558)
('位置', 0.003231749)
('非常', 0.0031730467)
('很棒', 0.003165329)
('飯店', 0.0031235393)
('我們', 0.003118872)
('一個', 0.0030242726)
('服務員', 0.0029957432)
('天成', 0.0029777628)
('人員', 0.0029738918)
('還有', 0.0029666321)
('服務', 0.002912789)
('再來', 0.0029079136)
('親切', 0.0028766452)
('不錯', 0.0028529959)
('態度', 0.0028144785)


In [179]:
ele=[]
for t in range(len(top)):
    ele.append(top[t][0])

In [180]:
select=range(ntopics)
for s in select:    
    top=lda.show_topic(s,30)
    ele=[]
    for t in range(len(top)):
        ele.append(top[t][0])   
    
    if s==select[0]:
        result=pd.DataFrame(ele,columns=[s])    
    else:         
        topics=pd.DataFrame(ele,columns=[s])       
        result = pd.concat([result, topics], axis=1)
result 

,0,1,2,3,4,5,6,7,8,9
0,非常,出口,酒店,車站,服務,台北,捷運,臺北,交通,台北
1,就是,員工,這家,台北,臺北,他們,交通,非常,飯店,這裏
2,酒店,台北,服務,入住,我們,這家,人員,友好,人員,酒店
3,乾淨,而且,交通,下次,酒店,一個,飯店,工作人員,我們,乾淨
4,舒適,飯店,這是,舒適,非常,捷运,服務,酒店,禮貌,下次
5,台北,推薦,火車站,服務,房間,酒店,天成,舒適,台北,位置
6,附近,酒店,可以,酒店,交通,一次,乾淨,我們,機場,非常
7,這家,交通,車站,整潔,下次,地鐵站,非常,旁邊,酒店,很棒
8,臺北,天成,方便,飯店,飯店,台灣,禮貌,車站,臺北,飯店
9,友好,可以,我們,方便,人員,服务,入住,早餐,服務,我們


In [181]:
# 第 n 個 topic
n=9
top = lda1.show_topic(n,20)
for t in top:
    print (t)

('下次', 0.0076292134)
('飯店', 0.006972398)
('台北', 0.0063674743)
('禮貌', 0.005997822)
('推薦', 0.005935608)
('交通', 0.005779005)
('人員', 0.005694419)
('捷運', 0.0054225577)
('整潔', 0.0053721233)
('入住', 0.005301197)
('服務員', 0.005190316)
('乾淨', 0.005139396)
('天成', 0.005055801)
('朋友', 0.0049538254)
('一定', 0.0049175494)
('非常', 0.0047769393)
('再來', 0.0047708442)
('服務態度', 0.0047192825)
('服務', 0.0044308985)
('親切', 0.0044205794)


In [182]:
ele=[]
for t in range(len(top)):
    ele.append(top[t][0])

In [183]:
select=range(ntopics)
for s in select:    
    top=lda1.show_topic(s,30)
    ele=[]
    for t in range(len(top)):
        ele.append(top[t][0])   
    
    if s==select[0]:
        result1=pd.DataFrame(ele,columns=[s])    
    else:         
        topics=pd.DataFrame(ele,columns=[s])       
        result1 = pd.concat([result1, topics], axis=1)
result1 

,0,1,2,3,4,5,6,7,8,9
0,如果,房间,那裡,我們,擔心,酒店,其他人,高铁,大部分,下次
1,左右,服务,算是,如果,大酒店,臺北,差不多,楼下,親朋好友,飯店
2,路程,车站,美中不足,強烈推薦,坐火車,服務,目的地,蛋糕,發現,台北
3,同事,捷运,隔音,大酒店,不用,火車站,樓層,洗澡,他們,禮貌
4,好離,干净,隔壁,一個,十分滿意,車站,禮物,好吃,我們,推薦
5,難忘,热情,我來,退房,人房,位置,商场,簽名,有關,交通
6,桃園,不错,這兒,沒有,當天,乾淨,美妙,除此之外,更好,人員
7,行李,人员,加強,手續,東西,舒適,那個,台铁,回到,捷運
8,周邊,旁边,當地,一次,走路,這家,有善,站點,一般,整潔
9,公共交通,选择,如果,房間,你們,台北,毛巾,美好,無與倫比,入住


In [184]:
lda.show_topics()

[(0,
  '0.003*"非常" + 0.003*"就是" + 0.003*"酒店" + 0.003*"乾淨" + 0.003*"舒適" + 0.003*"台北" + 0.003*"附近" + 0.003*"這家" + 0.003*"臺北" + 0.003*"友好"'),
 (1,
  '0.004*"出口" + 0.003*"員工" + 0.003*"台北" + 0.003*"而且" + 0.003*"飯店" + 0.003*"推薦" + 0.003*"酒店" + 0.003*"交通" + 0.003*"天成" + 0.003*"可以"'),
 (2,
  '0.004*"酒店" + 0.004*"這家" + 0.004*"服務" + 0.004*"交通" + 0.004*"這是" + 0.004*"火車站" + 0.004*"可以" + 0.004*"車站" + 0.003*"方便" + 0.003*"我們"'),
 (3,
  '0.004*"車站" + 0.004*"台北" + 0.004*"入住" + 0.004*"下次" + 0.004*"舒適" + 0.003*"服務" + 0.003*"酒店" + 0.003*"整潔" + 0.003*"飯店" + 0.003*"方便"'),
 (4,
  '0.003*"服務" + 0.003*"臺北" + 0.003*"我們" + 0.003*"酒店" + 0.003*"非常" + 0.003*"房間" + 0.003*"交通" + 0.003*"下次" + 0.003*"飯店" + 0.003*"人員"'),
 (5,
  '0.003*"台北" + 0.003*"他們" + 0.003*"這家" + 0.003*"一個" + 0.003*"捷运" + 0.003*"酒店" + 0.003*"一次" + 0.003*"地鐵站" + 0.003*"台灣" + 0.002*"服务"'),
 (6,
  '0.005*"捷運" + 0.004*"交通" + 0.004*"人員" + 0.004*"飯店" + 0.004*"服務" + 0.003*"天成" + 0.003*"乾淨" + 0.003*"非常" + 0.003*"禮貌" + 0.003*"入住"'),
 (7,
  '0.004*"臺北" + 0.00

In [185]:
import pyLDAvis.gensim
#pyLDAvis.gensim.prepare(lda, corpus_tfidf, dictionary)
pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus_tfidf, dictionary))

C:\Users\loy07\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [132]:
lda1.show_topics()

[(17,
  '0.006*"帶著" + 0.004*"食店" + 0.004*"需求" + 0.004*"樓有" + 0.004*"房型" + 0.003*"有關" + 0.003*"哪裡" + 0.003*"迎賓" + 0.003*"誠品" + 0.003*"床頭"'),
 (10,
  '0.008*"起來" + 0.006*"工作" + 0.006*"性價比" + 0.004*"客人" + 0.004*"洒店" + 0.003*"知道" + 0.003*"溢額" + 0.003*"上網" + 0.003*"要來" + 0.003*"新穎"'),
 (9,
  '0.009*"大饭店" + 0.007*"满意" + 0.007*"地铁" + 0.006*"亲切" + 0.006*"因为" + 0.006*"特别" + 0.005*"还是" + 0.005*"解決" + 0.005*"感觉" + 0.004*"服务"'),
 (2,
  '0.008*"鐵路" + 0.006*"臨近" + 0.004*"接受" + 0.004*"專門" + 0.003*"公車" + 0.003*"可說是" + 0.003*"上午" + 0.003*"哪裡" + 0.003*"上網" + 0.003*"我倆"'),
 (14,
  '0.008*"人房" + 0.006*"接近" + 0.006*"那裡" + 0.006*"那裏" + 0.004*"反應" + 0.004*"留下" + 0.004*"天花板" + 0.004*"給人" + 0.004*"廁所" + 0.004*"同行"'),
 (5,
  '0.006*"三口" + 0.004*"不遠" + 0.004*"來回" + 0.004*"後面" + 0.004*"齐全" + 0.004*"可能" + 0.003*"物品" + 0.003*"出来" + 0.003*"楼下" + 0.003*"走動"'),
 (15,
  '0.006*"按摩" + 0.006*"什麼" + 0.005*"可能" + 0.005*"和善" + 0.005*"樓下" + 0.004*"洗手間" + 0.004*"十分滿意" + 0.004*"不能" + 0.004*"設置" + 0.004*"重要"'),
 (19,
  '0.009*"

In [167]:
pyLDAvis.display(pyLDAvis.gensim.prepare(lda1, corpus_tfidf, dictionary))

C:\Users\loy07\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [154]:
vis_data=pyLDAvis.gensim.prepare(lda, corpus, dictionary,sort_topics=False)
pyLDAvis.save_html(vis_data,path+'hotelreview_lda_nopasses.html')

In [169]:
vis_data1=pyLDAvis.gensim.prepare(lda1, corpus, dictionary,sort_topics=False)
pyLDAvis.save_html(vis_data1,path+'hotelreview_lda_passes10.html')

C:\Users\loy07\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
